# VENDAS DE COMBUSTÍVEL NO BRASIL (ANP)

Script para extrair as bases de dados em https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/vendas-de-derivados-de-petroleo-e-biocombustiveis


Disponibilidades:
- Histórico
- Histórico por município

# HISTÓRICO

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

from IPython.display import clear_output

In [ ]:
html = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/vendas-de-derivados-de-petroleo-e-biocombustiveis"

page = requests.get(html)

soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
# LINKS

links_dados = []
for a in soup.find_all('a', href=True):
    if re.search(re.compile("dados-abertos", flags = re.IGNORECASE), str(a)):
        links_dados.append(a["href"])

links_historico = []
for link in links_dados:
    if re.search(re.compile("segmento"), str(link)):
        continue
    if re.search(re.compile("m3"), str(link)):
        links_historico.append(link)

In [ ]:
# DOWNLOADS

n = len(links_historico)
i = 0

combustiveis = pd.DataFrame()
for link in links_historico:
    df2 = pd.read_csv(link, sep = ";", decimal = ",")
    if len(df2.columns) == 18:
        df2 = df2.iloc[:,:-1]
    if len(df2.columns) == 19:
        df2 = df2.iloc[:,:-2]
    if not combustiveis.empty:
        colunas = combustiveis.columns
        df2.columns = colunas
    combustiveis = pd.concat([combustiveis, df2], axis = 0)
    clear_output(wait = True)
    i += 1
    print(f"{(i/n)*100}%")

In [ ]:
# TRATA NOME DAS COLUNAS, CASO HAJA ESPAÇO ANTES OU DEPOIS DAS PALAVRAS
colunas = []
for nome in combustiveis.columns:
    colunas.append(nome.strip())
combustiveis.columns = colunas

In [ ]:
# TRANSFORMA FORMATO WIDE EM LONG
combustiveis_melted = pd.melt(combustiveis, id_vars = combustiveis.columns[:5], value_vars = combustiveis.columns[5:], var_name="MES", value_name="VENDAS")

In [ ]:
# FUNÇÃO PARA JUNTAR COLUNA "ANO" E "MES" EM DATA

def transforma_data(x):
    
    mes_dict = {
    'JANEIRO': "01",
    'FEVEREIRO': "02",
    'MARÇO': "03",
    'ABRIL': "04",
    'MAIO': "05",
    'JUNHO': "06",
    'JULHO': "07",
    'AGOSTO': "08",
    'SETEMBRO': "09",
    'OUTUBRO': "10",
    'NOVEMBRO': "11",
    'DEZEMBRO': "12",
    }
    
    data = pd.to_datetime(str(x["ANO"]) + "-" + mes_dict[x["MES"]] + "-" + "01", format = "%Y-%m-%d")
    return data
    

combustiveis_melted["DATA"] = combustiveis_melted.apply(transforma_data, axis = 1)
combustiveis_anp = combustiveis_melted[["COMBUSTÍVEL", "REGIÃO", "ESTADO", "DATA", "VENDAS"]].dropna()
combustiveis_uf = combustiveis_anp.sort_values(by = ["COMBUSTÍVEL", "ESTADO", "DATA"])

In [ ]:
combustiveis_uf.head(6)

# MUNICÍPIOS

In [ ]:
links_municipios = []
for link in links_dados:
    if re.search(re.compile("segmento"), str(link)):
        continue
    if re.search(re.compile("municipio"), str(link)) and re.search(re.compile(".csv"), str(link)):
        links_municipios.append(link)

In [ ]:
n = len(links_municipios)
i = 0

combustiveis = pd.DataFrame()
for link in links_municipios:
    clear_output(wait = True)    
    try:
        df2 = pd.read_csv(link, sep = ";", decimal = ",")
    except:
        df2 = pd.read_csv(link, sep = ";", decimal = ",", encoding = "latin-1")
    if not combustiveis.empty:
        colunas = combustiveis.columns
        df2.columns = colunas
    combustiveis = pd.concat([combustiveis, df2], axis = 0)
    i += 1
    print(f"{round((i/n)*100,2)}%")
    
combustiveis.loc[:, "Vendas"] = combustiveis["Vendas"].str.replace(".", "")

In [ ]:
combustiveis.head(6)